In [3]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [4]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [10]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://f729-34-122-240-65.ngrok.io'

qs_res_body = []
qs_res_title = []
qs_res_anchor = []
qs_res_rank = []
qs_res_views = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res_body = requests.get(url + '/search_body', {'query': q}, timeout=35)
    duration_body = time() - t_start
    t_start=time()
    res_title = requests.get(url + '/search_title', {'query': q}, timeout=35)
    duration_title = time() - t_start
    t_start=time()
    res_anchor = requests.get(url + '/search_anchor', {'query': q}, timeout=35)
    duration_anchor = time() - t_start
    if res_body.status_code == 200:
      pred_wids, _ = zip(*res_body.json())
      ap_body = average_precision(true_wids, pred_wids)
    # if res_title.status_code == 200:
    #   pred_wids, _ = zip(*res_title.json())
    #   ap_title = average_precision(true_wids, pred_wids)
    # if res_anchor.status_code == 200:
    #   pred_wids, _ = zip(*res_anchor.json())
    #   ap_anchor = average_precision(true_wids, pred_wids)
    
  except:
    pass
  
  qs_res_body.append((q, duration_body, ap_body))
  # qs_res_title.append((q, duration_title, ap_title))
  # qs_res_anchor.append((q, duration_anchor, ap_anchor))
print(qs_res_body)
# print(qs_res_title)
# print(qs_res_anchor)
#print(len(qs_res))

#search test
#print(requests.get(url + '/search',{'query':'hello'}, timeout=35).text)

#get_pageview test
#page id = 	675561 ????? elbaz search what is this this page
#print(requests.post(url + '/get_pageview', json=[12,675561,8,2023]).text)
#print(requests.get(url + '/search_anchor', {'query': 'political'}, timeout=35).text)
#print(requests.get(url + '/search_anchor', {'query': 'Clinical'}, timeout=35).text)
# print(requests.get(url + '/search_title', {'query': 'yoga'}, timeout=35).text)
# print(requests.get(url + '/search_title', {'query': 'NBA'}, timeout=35).text)


[('python', 0.13433003425598145, 0.737), ('data science', 0.12847399711608887, 0.187), ('migraine', 0.18093514442443848, 0.902), ('chocolate', 0.19231295585632324, 0.236), ('how to make pasta', 0.11969518661499023, 0.209), ('Does pasta have preservatives?', 0.19460463523864746, 0.955), ('how google works', 0.11879777908325195, 0.103), ('what is information retrieval', 0.117156982421875, 0.957), ('NBA', 0.19048404693603516, 0.978), ('yoga', 0.11332058906555176, 0.821), ('how to not kill plants', 0.11832809448242188, 0.589), ('masks', 0.11697721481323242, 0.781), ('black friday', 0.12832975387573242, 0.895), ('why do men have nipples', 0.11750459671020508, 1.0), ('rubber duck', 0.20731711387634277, 0.969), ('michelin', 0.13570523262023926, 0.711), ('what to watch', 0.11944460868835449, 0.229), ('best marvel movie', 0.1269826889038086, 0.821), ('how tall is the eiffel tower', 0.11850214004516602, 0.868), ('where does vanilla flavoring come from', 0.12590694427490234, 0.489), ('best ice cr

In [27]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://0759-34-122-240-65.ngrok.io'

t_start = time()
res = requests.post(url + '/get_pageview', json=[34258, 59580357, 315141, 47035547, 626718, 32016617, 59516988, 61583621, 38424, 196789, 734272, 60851303, 988753, 92222, 419015, 60992198, 621809, 83742, 3127300, 23834216, 1381751, 744158, 18911094, 1817874, 59052395, 43562594, 31793047, 6154795, 17452177, 62342792, 13936750, 60805249, 60823333, 59899027, 46719542, 7702313, 60310807, 44270128, 68489731, 60500106, 1652601, 359983, 60300514, 45482242, 34756533, 58520592, 43611227, 47586513, 1775374, 6933106, 13412771, 62549131, 49278248, 62557135, 4523354, 60468426, 666420, 1017009, 35795560, 34843274, 60199413, 16488467, 29828807, 585681, 41757168, 66018807, 61285582, 1226448, 28848113, 11487904, 4242777, 1661867, 10671559, 44035836, 6931929, 60705564, 61690747, 7343803, 9042644, 36991237, 3965223, 63090179, 632990, 1041815, 53087254, 418999, 1632919])
duration = time() - t_start
print(duration)
print(res.text)

t_start = time()
res = requests.post(url + '/get_pagerank', json=[34258, 59580357, 315141, 47035547, 626718, 32016617, 59516988, 61583621, 38424, 196789, 734272, 60851303, 988753, 92222, 419015, 60992198, 621809, 83742, 3127300, 23834216, 1381751, 744158, 18911094, 1817874, 59052395, 43562594, 31793047, 6154795, 17452177, 62342792, 13936750, 60805249, 60823333, 59899027, 46719542, 7702313, 60310807, 44270128, 68489731, 60500106, 1652601, 359983, 60300514, 45482242, 34756533, 58520592, 43611227, 47586513, 1775374, 6933106, 13412771, 62549131, 49278248, 62557135, 4523354, 60468426, 666420, 1017009, 35795560, 34843274, 60199413, 16488467, 29828807, 585681, 41757168, 66018807, 61285582, 1226448, 28848113, 11487904, 4242777, 1661867, 10671559, 44035836, 6931929, 60705564, 61690747, 7343803, 9042644, 36991237, 3965223, 63090179, 632990, 1041815, 53087254, 418999, 1632919])
duration = time() - t_start
print(duration)
print(res.text)


0.11990141868591309
[80260,7586,24309,5847,24350,6509,1831,1489,10675,7806,19795,22388,14796,17410,16266,843,23057,10667,1376,2092,11403,7627,8427,3783,1244,2328,1334,1828,5328,241,2943,54,380,213,15523,5708,705,10425,1439,941,908,8043,133,215,1163,656,22118,560,720,5019,1498,59,825,132,607,724,3693,2012,1054,5773,190,1504,338,12343,1639,449,61,4850,522,408,2531,969,553,674,963,585,184,268,60,582,1053,922,15679,6257,120,5183,15579]

0.0966634750366211
[83.99594185114448,9.45604981472203,8.380580114251691,8.11020896045297,16.272385307418013,0.41291724286653814,1.7940347142121646,0.26277566427191196,1.2218084388647872,1.7393116070195953,2.6741742356334357,1.7221327530291795,1.575401332613651,2.97067615767833,3.730605548508708,0.6928982299460693,11.194186005258684,13.65855912454737,1.711961906484961,1.1245083820805095,5.515282580218201,6.287814772813479,0.47042904830792065,1.045163072707094,2.446854098995287,0.2422050679132725,0.5457835440994311,1.2202916730566309,0.9985459792437439,0.184

In [45]:
# import matplotlib.pyplot as plt
# q = [i[0] for i in qs_res]
# x = [i[1] for i in qs_res]
# y = [i[2] for i in qs_res]


# plt.bar(x, y, tick_label = q,width = 0.8, color = ['red', 'green'])
# plt.xlabel("query")
# plt.ylabel("duration")
# #plt.ylabel("average precision")

# plt.title("first test - durations")

# plt.show()


save_test_list = qs_res
print(save_test_list)

[('python', 15.070565938949585, 1.0), ('data science', 17.99678111076355, 0.968), ('migraine', 7.277550458908081, 1.0), ('chocolate', 20.199448585510254, 0.955), ('how to make pasta', 3.704864263534546, 1.0), ('Does pasta have preservatives?', 3.7639801502227783, 0.2), ('how google works', 27.856329679489136, 0.946), ('what is information retrieval', 10.921855449676514, 0.989), ('NBA', 15.17752981185913, 1.0), ('yoga', 25.972076177597046, 1.0), ('how to not kill plants', 0.07571816444396973, None), ('masks', 6.999579906463623, 1.0), ('black friday', 23.65346097946167, 0.998), ('why do men have nipples', 0.09122037887573242, None), ('rubber duck', 4.815493106842041, 0.803), ('michelin', 8.729891777038574, 1.0), ('what to watch', 7.20874547958374, 1.0), ('best marvel movie', 19.271244525909424, 1.0), ('how tall is the eiffel tower', None, None), ('where does vanilla flavoring come from', None, None), ('best ice cream flavour', 15.673341512680054, 0.992), ('how to tie a tie', 10.267841577